In [45]:
import sys
sys.path
sys.path.append('C:\\Users\\36541\\anaconda3\\envs\\openai\\Lib\\site-packages')

In [46]:
sys.path

['D:\\360_study_materials\\MARL\\Path_to_MARL\\openAI',
 'C:\\Users\\36541\\anaconda3\\python311.zip',
 'C:\\Users\\36541\\anaconda3\\DLLs',
 'C:\\Users\\36541\\anaconda3\\Lib',
 'C:\\Users\\36541\\anaconda3',
 '',
 'C:\\Users\\36541\\anaconda3\\Lib\\site-packages',
 'C:\\Users\\36541\\anaconda3\\Lib\\site-packages\\win32',
 'C:\\Users\\36541\\anaconda3\\Lib\\site-packages\\win32\\lib',
 'C:\\Users\\36541\\anaconda3\\Lib\\site-packages\\Pythonwin',
 'C:\\Users\\36541\\anaconda3\\envs\\openai\\Lib\\site-packages']

<h2>RL with FrozenLake</h2>
<p>Fristly we will use q-learning to control and improve every move of agent</p>

In [1]:
import gymnasium as gym
import numpy as np
from IPython.display import clear_output
env= gym.make("FrozenLake-v1",render_mode="human")#env
#create q-table
state_size = env.observation_space.n
action_size = env.action_space.n
q_table = 0.1*np.random.random([state_size, action_size])
q_table

array([[0.02759273, 0.05130485, 0.03214128, 0.02951271],
       [0.09305312, 0.07173051, 0.01503383, 0.04451518],
       [0.0300752 , 0.04431072, 0.02050409, 0.02178363],
       [0.04618061, 0.04253456, 0.04181581, 0.00873054],
       [0.02305321, 0.06014916, 0.04236924, 0.01012737],
       [0.00973318, 0.02093988, 0.01116267, 0.0275529 ],
       [0.08054146, 0.06344372, 0.05124157, 0.01594153],
       [0.04657376, 0.03667984, 0.05996925, 0.07972934],
       [0.05119087, 0.0459588 , 0.00527591, 0.05975778],
       [0.06172145, 0.04544333, 0.08692103, 0.02420912],
       [0.03606536, 0.05117257, 0.07453123, 0.04611979],
       [0.05374448, 0.02356084, 0.0621444 , 0.07672921],
       [0.06306768, 0.03071108, 0.04899696, 0.0181361 ],
       [0.07615041, 0.09915665, 0.0705638 , 0.03554788],
       [0.00931007, 0.01661018, 0.07431083, 0.02133869],
       [0.07476326, 0.02660031, 0.04811081, 0.05644341]])

<p>Now we need to train this:</p>
<ol>
    <li>get the action according to q-table </li>
    <li>observe state and rewards </li>
    <li>update the q-value in q-table </li>
</ol>

In [3]:
env= gym.make("FrozenLake-v1",render_mode="human")
observation, info = env.reset(seed=42)
q_table = np.random.random([state_size, action_size])
for _ in range(100):
    #1
    action = np.argmax(q_table[observation])
    observation_now=observation
    #2
    observation, reward, terminated, truncated, info = env.step(action)
    #3
    q_table[observation_now][action]+=0.1*(reward+0.7*np.max(q_table[observation])-q_table[observation_now][action])
    print(q_table)
    clear_output(wait=True)
    if terminated or truncated:
        observation, info = env.reset()
env.close()

[[0.57868612 0.17063301 0.57816984 0.56492707]
 [0.56241247 0.7611826  0.74216403 0.02544439]
 [0.69139307 0.68942938 0.14163453 0.67544662]
 [0.45983983 0.13149908 0.58262424 0.67535692]
 [0.18806529 0.06205141 0.60543356 0.04992693]
 [0.95742754 0.51994768 0.01063152 0.66318695]
 [0.7746028  0.20950859 0.7528882  0.1214022 ]
 [0.30026485 0.20718593 0.63559762 0.60472507]
 [0.24804659 0.06865012 0.58080306 0.57502082]
 [0.06095895 0.49095874 0.51295914 0.34032159]
 [0.60754184 0.66020999 0.64469276 0.28783784]
 [0.14555165 0.27118272 0.38254015 0.59337237]
 [0.05467195 0.34512906 0.167713   0.32681001]
 [0.13351706 0.08148994 0.27467398 0.01590998]
 [0.50940107 0.72243015 0.74240629 0.2200136 ]
 [0.7586152  0.60253639 0.25563786 0.89016835]]


<p>Oh,man,it doesn't improve themselves too much, what to do????</p>
<p>I think we can try to make some exploration by using &epsilon;-greedy</p>

In [24]:
import random
epsilon = 1.0001  
epsilon_decay_rate = 0.0001  
def get_action(observation,q_table,env):
    global epsilon
    epsilon-= 0.0001  
    action_greedy = np.argmax(q_table[observation])
    action_random = env.action_space.sample()
    return action_random if random.random() < epsilon else action_greedy

In [18]:
env= gym.make("FrozenLake-v1",is_slippery=False,render_mode="human")
observation, info = env.reset(seed=42)
state_size = env.observation_space.n
action_size = env.action_space.n
q_table = np.zeros([state_size, action_size])
learning_rate_a = 0.9
discount_factor_g = 0.9 
for _ in range(10000):
    #1
    action = get_action(observation,q_table,env)
    observation_now = observation
    #2
    observation, reward, terminated, truncated, info = env.step(action)
    #3
    q_table[observation_now][action]+=learning_rate_a*(reward+learning_rate_a*np.max(q_table[observation])-q_table[observation_now][action])
    print(q_table)
    clear_output(wait=True)
    if terminated or truncated:
        observation, info = env.reset()
env.close()

[[0.531441 0.59049  0.59049  0.531441]
 [0.531441 0.       0.6561   0.59049 ]
 [0.59049  0.729    0.59049  0.6561  ]
 [0.6561   0.       0.59049  0.59049 ]
 [0.59049  0.6561   0.       0.531441]
 [0.       0.       0.       0.      ]
 [0.       0.81     0.       0.6561  ]
 [0.       0.       0.       0.      ]
 [0.6561   0.       0.729    0.59049 ]
 [0.6561   0.81     0.81     0.      ]
 [0.729    0.9      0.       0.729   ]
 [0.       0.       0.       0.      ]
 [0.       0.       0.       0.      ]
 [0.       0.81     0.9      0.729   ]
 [0.81     0.9      1.       0.81    ]
 [0.       0.       0.       0.      ]]


In [19]:
env= gym.make("FrozenLake-v1",is_slippery=False,render_mode="human")
observation, info = env.reset(seed=42)
for _ in range(30):
    #1
    action = get_action(observation,q_table,env)
    
    observation_now = observation
    #2
    observation, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        observation, info = env.reset()
env.close()

<p>Result looks great, q-learning theory does work!</p>
<p>I found that the initial setting of the Q-table is important; it can change the update direction of the whole process >However,the update rate is slow.</p>
<p>I think we can try using neural networks to see if we can change that. We are going to use the famous paper "<strong>Human-level control through deep reinforcement learning</strong>" as our algorithm.</
<p>But first let's start with a simple neural network</p>

<h2>simple neural networks</h2>

In [20]:
import torch
import torch.nn as nn
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super().__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.layer_int = nn.Linear(state_size+action_size, 64)
        self.layer_out = nn.Linear(64, 1)
    def forward(self, inputs):
        q_values = self.layer_int(inputs)
        q_values = self.layer_out(q_values)
        #action = torch.argmax(q_values).item()
        return q_values#.detach().numpy()
        

In [21]:
#one-hot encoding
def state_to_dqn_input( state:int, num_states:int,action:int,num_action:int)->torch.Tensor:
        input_tensor = torch.zeros(num_states+num_action)
        input_tensor[state] = 1
        input_tensor[num_states+action] = 1
        return input_tensor

In [33]:
#check model's feature
model = QNetwork(16,4)
observation_now=1
action=3
one_hot_intputs = state_to_dqn_input(observation_now,16,action,4)
one_hot_intputs

tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1.])

In [35]:
env= gym.make("FrozenLake-v1",render_mode="human")
observation, info = env.reset(seed=42)
state_size = env.observation_space.n
action_size = env.action_space.n
q_table = np.zeros([state_size, action_size])
learning_rate_a = 0.9
discount_factor_g = 0.9
loss_fn = nn.MSELoss() 
model = QNetwork(16,4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for _ in range(1000):
    #1
    epsilon = 1
    action_greedy = np.argmax(q_table[observation])
    action_random = env.action_space.sample()
    action = action_random if random.random() < epsilon else action_greedy
    epsilon-= 0.001  
    observation_now = observation
    #2
    observation, reward, terminated, truncated, info = env.step(action)
    #3
    one_hot_intputs = state_to_dqn_input(observation_now,16,action,4)
    model.train()
    q_value=model(one_hot_intputs)    
    q_target= q_value+\
                  learning_rate_a*(reward+discount_factor_g*(1-terminated)*np.max(q_table[observation])-q_value)
    
    q_table[observation_now][action]=q_target
    q_target= torch.tensor([q_target],dtype=torch.float32)
    loss= loss_fn(q_value,q_target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(q_table)
    clear_output(wait=True)
    if terminated or truncated:
        observation, info = env.reset()
env.close()

[[ 3.68488133e-01  3.98570210e-01  3.74067187e-01  4.11178410e-01]
 [ 2.85750031e-02  4.40580338e-01  4.51021135e-01  3.61900687e-01]
 [ 5.16256452e-01  3.78689378e-01  5.26460767e-01  3.53765815e-01]
 [ 3.80863070e-01  1.74489021e-02  2.26877064e-01  3.37339193e-01]
 [ 2.83553153e-01  4.80325937e-01  3.86109769e-01  2.86018074e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 4.82944548e-02  1.85503811e-03  4.64841962e-01  3.68379056e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 3.58521521e-01  1.66386453e-05  4.01314318e-01  9.69863776e-03]
 [ 2.84698904e-02  6.88004494e-01  4.32265878e-01  6.91982806e-01]
 [ 4.69033539e-01  4.07519996e-01  7.91693568e-01  6.07887506e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 3.89754325e-01  3.58977914e-02  4.28396404e-01 -6.01392612e-03]
 [ 6.53937459e-01  7.82589078e-01  9.00001884e-01  6.98756397e

In [37]:
env= gym.make("FrozenLake-v1",is_slippery=False,render_mode="human")
observation, info = env.reset(seed=42)
for _ in range(30):
    #1
    action = get_action(observation,q_table,env)
    
    observation_now = observation
    #2
    observation, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        observation, info = env.reset()
env.close()

<p>Ehhh.... What? it is even wrose!</p>
<p>I try to use a neural netwerk to predict q-value, and use it to update the q-table, but it obviously doesn't work out</p>
<p>I'd say let's check what the paper said about the netwerk model</p>
It says we will use double netwerks; one for predict q-value, other for target value(compare with the frist one)
And also experience replay is used for efficiency

<h2>Human-level control through deep reinforcement 
learning</h2>

In [91]:
import gymnasium as gym
import numpy as np
from collections import deque
import random
import torch
from torch import nn
import torch.nn.functional as F

class DQN(nn.Module):
    def __init__(self, state_size, action_size):
        super().__init__()
        self.layer_int = nn.Linear(state_size, 64)
        self.layer_out = nn.Linear(64, action_size)
    def forward(self, inputs):
        q_values = self.layer_int(inputs)
        q_values = self.layer_out(q_values)
        #action = torch.argmax(q_values).item()
        return q_values#.detach().numpy()
def state_to_dqn_input( state:int, num_states:int)->torch.Tensor:
        input_tensor = torch.zeros(num_states)
        input_tensor[state] = 1
        return input_tensor

In [44]:
observation = state_to_dqn_input(1, 16)
new_observation = state_to_dqn_input(2, 16)
new_observation_1 = state_to_dqn_input(3, 16)
a= np.asarray([observation,new_observation,new_observation_1])
a= torch.as_tensor(a,dtype=torch.float32)
action=torch.tensor([1,2, 3])
model = DQN(16,4)
b=model(a)
print(b.gather(dim=-1,index=action.view(-1,1)))

tensor([[ 0.1352],
        [-0.1024],
        [-0.2146]], grad_fn=<GatherBackward0>)


In [ ]:
#parameter
EPOCH=1000
learning_rate_a = 0.001         
discount_factor_g = 0.9        
network_sync_rate = 10         
replay_memory_size = 1000      
mini_batch_size = 32
epsilon = 1
#enviorment
env= gym.make("FrozenLake-v1",is_slippery=False,render_mode="human")
#experience buffer (s_i,a,r,s_i+1,done)
memory = deque([], maxlen=100)
#double model,loss,optimizer
loss_fn = nn.MSELoss()
train_model = DQN(16,4)
target_model = DQN(16,4)
target_model.load_state_dict(train_model.state_dict())
optimizer  = torch.optim.Adam(train_model.parameters(), lr=learning_rate_a)
#epsilon is combined with epochs
epochs = np.linspace(0,1,EPOCH)
for i in range(EPOCH):
    observation, info = env.reset(seed=42)
    terminated = False
    truncated  = False
    reward_count = 0
    while(not reward_count>=40):
        if random.random() < epsilon:
            # select random action
            action = env.action_space.sample() # actions: 0=left,1=down,2=right,3=up
        else:         
            with torch.no_grad():
                action = train_model(state_to_dqn_input(observation, 16)).argmax().item()
        # add to memory
        new_observation, reward, terminated, truncated, info = env.step(action)
        observation_t = state_to_dqn_input(observation, 16)
        new_observation_t = state_to_dqn_input(new_observation, 16)
        memory.append((observation_t, action , reward, new_observation_t,terminated)) 
        observation = new_observation
        reward_count+=1#reward
        if terminated or truncated:
            observation, info = env.reset()
    if i%100==0:
        print('got the gift!'+f'{i} times')
    if len(memory)>mini_batch_size:
        mini_barch = random.sample(memory, mini_batch_size)
        
        obs_a = np.asarray([t[0] for t in mini_barch])
        action_a = np.asarray([t[1] for t in mini_barch])
        reward_a = np.asarray([t[2] for t in mini_barch])
        new_obs_a = np.asarray([t[3] for t in mini_barch])
        terminated_a = np.asarray([t[4] for t in mini_barch])
        
        obs_t = torch.as_tensor(obs_a,dtype=torch.float32)
        action_t = torch.as_tensor(action_a,dtype=torch.int64).view(-1,1)
        reward_t = torch.as_tensor(reward_a,dtype=torch.float32).view(-1,1)
        new_obs_t = torch.as_tensor(new_obs_a,dtype=torch.float32)
        terminated_t = torch.as_tensor(terminated_a,dtype=torch.float32).view(-1,1)
        #compute the target
        max_q_next_action = target_model(new_obs_t).max(dim=-1,keepdim=True)[0]
        target = reward_t+ (1-terminated_t)*discount_factor_g*max_q_next_action
        #compute the loss
        q_values = train_model(obs_t).gather(dim=-1,index=action_t)
        loss = loss_fn(q_values,target)
        #optimizer
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #update
        epsilon = max(epsilon - 1/EPOCH, 0)
        if i % network_sync_rate==0:
            target_model.load_state_dict(train_model.state_dict())
env.close()
torch.save(train_model.state_dict(), "frozen_lake_dql.pt")

<p>CPU is way to slow to compute, i'd suggest you to use google colab where you can use free GPU</p>
<p>so i just train the model, download the state_dict file and load it in here</p>

In [108]:
load_model=DQN(16,4)
MODEL_SAVE_PATH='D:\\360_study_materials\\MARL\\Path_to_MARL\\openAI\\frozen_lake_dql.pt'
stae_dict=torch.load(MODEL_SAVE_PATH, map_location=torch.device('cpu'), weights_only=True)
load_model.load_state_dict(stae_dict)

<All keys matched successfully>

In [112]:
env= gym.make("FrozenLake-v1",is_slippery=False,render_mode="human")
observation, info = env.reset(seed=42)
for _ in range(30):
    #1
    observation_t = state_to_dqn_input(observation, 16)
    q_values = load_model(observation_t)
    action =torch.argmax(q_values).item()
    
    #2
    observation, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        observation, info = env.reset()
env.close()

<p>Wonderful! The reult looks great!</p>
<h3>Summarize</h3>
<ul>
    <li>create a Experience Replay buffer </li>
    <li>choose an action according to epsilon-greedy</li>
    <li>save the infomation to buffer</li>
    <li>feed the batch to netwerk</li>
    <li>compute the target q-value</li>
    <li>train the netwerk:loss.backward(),optimizer.step()...</li>
    <li>update the target netwerk every C steps</li>
</ul>